In [1]:
import pandas
import geopandas 
import matplotlib.pyplot as plt
from   matplotlib.pyplot import imread
import matplotlib.patches as mpatches
import numpy as np
import cartopy.crs as ccrs
import cartopy.feature as cfeat
#import cartopy.io.shapereader as shpreader
from   cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

import shapely.geometry as sgeom
from   shapely.geometry import LineString
from   shapely.geometry import MultiPoint
from   shapely.geometry import Point, Polygon

%matplotlib inline

In [2]:
import urllib
import requests
from   zipfile import ZipFile
import datetime 



In [7]:
Advisory_Forecast_Track_Cone_of_Uncertainty_and_Watches_Warnings = 'https://www.nhc.noaa.gov/gis/forecast/archive/al122021_5day_latest.zip'
Advisory_Wind_Field_and_Forecast_Wind_Radii                      = 'https://www.nhc.noaa.gov/gis/forecast/archive/al122021_fcst_latest.zip'
Preliminary_Best_Track                                           = 'https://www.nhc.noaa.gov/gis/best_track/al122021_best_track.zip'
Graphical_Tropical_Weather_Outlook                               = 'https://www.nhc.noaa.gov/xgtwo/gtwo_shapefiles.zip'
Wind_Speed_Probabilities_Points                                  = 'https://www.nhc.noaa.gov/gis/forecast/archive/wsp_120hrhalfDeg_latest.zip'
Wind_Speed_Probabilities_Polygons                                = 'https://www.nhc.noaa.gov/gis/forecast/archive/wsp_120hr5km_latest.zip'

In [4]:
file_name = 'gtwo_shapefiles.zip'
url = f'https://www.nhc.noaa.gov/xgtwo/{file_name}'
# download the file contents in binary format
r = requests.get(url)
# open method to open a file on your system and write the contents
with open(f'nhc_latest/{file_name}', 'wb') as code:
    code.write(r.content)
with ZipFile(f'nhc_latest/{file_name}', 'r') as gtwo_zip:
    #print(gtwo_zip.namelist())
    gtwo_areas = gtwo_zip.namelist()[2]
    gtwo_lines = gtwo_zip.namelist()[7]
    gtwo_points = gtwo_zip.namelist()[12]
gtwo_areas_gdf  = geopandas.read_file(f'zip://./nhc_latest/{file_name}!{gtwo_areas}')
gtwo_lines_gdf  = geopandas.read_file(f'zip://./nhc_latest/{file_name}!{gtwo_lines}')
gtwo_points_gdf = geopandas.read_file(f'zip://./nhc_latest/{file_name}!{gtwo_points}')

In [5]:
storm_number = '12'
current_year = datetime.date.today().year
file_names   = [
                f'al{storm_number}{current_year}_5day_latest.zip',
                f'al{storm_number}{current_year}_fcst_latest.zip',
                f'al{storm_number}{current_year}_best_track.zip',
                f'gtwo_shapefiles.zip',
                f'wsp_120hrhalfDeg_latest.zip',
                f'wsp_120hr5km_latest.zip'
               ]
urls         = [
                f'https://www.nhc.noaa.gov/gis/forecast/archive',
                f'https://www.nhc.noaa.gov/gis/forecast/archive',
                f'https://www.nhc.noaa.gov/gis/best_track',
                f'https://www.nhc.noaa.gov/xgtwo',
                f'https://www.nhc.noaa.gov/gis/forecast/archive',
                f'https://www.nhc.noaa.gov/gis/forecast/archive'
               ]
gdf_names   =  {
                file_names[0] : ['track_line_gdf', 'cone_gdf', 'points_gdf'],
                file_names[1] : ['init_radii_gdf', 'fcst_radii_gdf'],
                file_names[2] : ['best_track_points_gdf', 'best_track_line_gdf', 'best_track_radii_gdf', 'best_track_swath_gdf'],
                file_names[3] : ['gtwo_areas_gdf', 'gtwo_lines_gdf', 'gtwo_points_gdf'],
                file_names[4] : ['wsp_34_gdf_points', 'wsp_50_gdf_points', 'wsp_64_gdf_points'],
                file_names[5] : ['wsp_34_gdf_polygons', 'wsp_50_gdf_polygons', 'wsp_64_gdf_polygons']
               }


In [6]:
for idx, file_name in enumerate(file_names):
    url = f'{urls[idx]}/{file_name}'
    r = requests.get(url)
    with open(f'nhc_latest/{file_name}', 'wb') as code:
        code.write(r.content) 
_  = [0, 3, 4, 5]  
__ = [2, 7, 12]  
for idx, file_name in enumerate([file_names[0],  file_names[3], 
                                file_names[4],  file_names[5]]): 
    with ZipFile(f'nhc_latest/{file_name}', 'r') as zip_file_name:
        for idx2 in range(len(gdf_names[file_names[_[idx]]])):
            gdf_names[file_names[_[idx]]][idx2]  = geopandas.read_file(
            f'zip://./nhc_latest/{file_name}!{zip_file_name.namelist()[__[idx2]]}')
_  = [0,5]                                 
with ZipFile(f'nhc_latest/{file_names[1]}', 'r') as zip_file_name:
        for idx in range(2):
            gdf_names[file_names[1]][idx]  = geopandas.read_file(
            f'zip://./nhc_latest/{file_names[1]}!{zip_file_name.namelist()[_[idx]]}')
_  = [2, 7, 13, 16]                                 
with ZipFile(f'nhc_latest/{file_names[2]}', 'r') as zip_file_name:
        for idx in range(4):
            gdf_names[file_names[2]][idx]  = geopandas.read_file(
            f'zip://./nhc_latest/{file_names[2]}!{zip_file_name.namelist()[_[idx]]}')
        
        
   
gdf_names[file_names[2]][3].head(3)

,RADII,STORMID,BASIN,STORMNUM,ADVNUM,SYNOPTIME,TIMEZONE,NE,SE,SW,NW,geometry
0,34.0,AL122021,al,12.0,13.0,2021090100,UTC,0.0,0.0,0.0,50.0,"POLYGON ((-22.50000 11.90000, -22.50000 11.900..."
1,34.0,AL122021,al,12.0,13.0,2021090106,UTC,0.0,0.0,50.0,50.0,"POLYGON ((-24.50000 12.20000, -24.50000 12.200..."
2,34.0,AL122021,al,12.0,13.0,2021090112,UTC,50.0,50.0,40.0,0.0,"POLYGON ((-26.49254 13.23181, -26.47763 13.231..."


In [7]:
for idx, file_name in enumerate(file_names):
    url = f'{urls[idx]}/{file_name}'
    
    # download the file contents in binary format
    r = requests.get(url)
    # open method to open a file on your system and write the contents
    with open(f'nhc_latest/{file_name}', 'wb') as code:
        code.write(r.content) 
    with ZipFile(f'nhc_latest/{file_name}', 'r') as zip_file_name:
        if file_name in ['al122021_5day_latest.zip', 'gtwo_shapefiles.zip', 'wsp_120hrhalfDeg_latest.zip', 
                         'wsp_120hr5km_latest.zip']:
            pass
        
        shape_file  = zip_file_name.namelist()[2]
        elif file_name == 'al122021_fcst_latest.zip':
            pass
        else:
            pass
            

SyntaxError: invalid syntax (1536179167.py, line 15)

In [1]:
import feedparser
url = 'https://www.nhc.noaa.gov/index-at.xml'
f = feedparser.parse(url)

In [13]:
#f
#f.keys()
# len(f.entries)
# f['entries'][1]['nhc_atcf']
# f.status
# f.feed
# f.feed['subtitle']
# f.entries
# type(f.entries)
# for entry in f.entries:
#     print(entry.title)

dict_keys(['bozo', 'entries', 'feed', 'headers', 'etag', 'updated', 'updated_parsed', 'href', 'status', 'encoding', 'version', 'namespaces'])

In [17]:
df = pandas.DataFrame(f.entries)
df = df.drop(columns=['title_detail', 'summary', 'summary_detail', 'published_parsed', 'links', 'link', 'id', 
                 'guidislink', 'authors', 'author', 'author_detail'])
df.head(1)

,title,published,nhc_center,nhc_type,nhc_name,nhc_wallet,nhc_atcf,nhc_datetime,nhc_movement,nhc_pressure,nhc_wind,nhc_headline,nhc_cyclone
0,Summary for Hurricane Larry (AT2/AL122021),"Sun, 05 Sep 2021 20:49:08 GMT","20.5, -50.6",Hurricane,Larry,AT2,AL122021,5:00 PM AST Sun Sep 5,NW at 13 mph,955 mb,125 mph,...LARRY REMAINS A LARGE MAJOR HURRICANE... .....,


In [7]:
df['nhc_atcf']
l = [i for i in df['nhc_atcf'] if pandas.isnull(i) == False] 
l = [i[2:4] for i in l]
l

['12']

In [22]:
#{key: value for (key, value) in iterable}
d = {nhc_name: [nhc_atcf    , nhc_type, nhc_center, nhc_movement, 
                nhc_pressure, nhc_wind, published , nhc_datetime]         
     for (nhc_name    , nhc_atcf, nhc_type  , nhc_center  , nhc_movement,
          nhc_pressure, nhc_wind, published , nhc_datetime) 
     in  zip(df['nhc_name']    , df['nhc_atcf']    , df['nhc_type'], df['nhc_center'], 
             df['nhc_movement'], df['nhc_pressure'], df['nhc_wind'], df['published'],
             df['nhc_datetime']) 
     if pandas.isnull(nhc_name) == False}   
d

{'Larry': ['AL122021',
  'Hurricane',
  '20.5, -50.6',
  'NW at 13 mph',
  '955 mb',
  '125 mph',
  'Sun, 05 Sep 2021 20:49:08 GMT',
  '5:00 PM AST Sun Sep 5']}

In [25]:
d['Larry'][0][2:4]

'12'

In [19]:
storm_number = '12'
current_year = datetime.date.today().year

def nhc_fnames(storm_number = '12', year = current_year):
    file_names   = [
                    f'al{storm_number}{year}_5day_latest.zip',
                    f'al{storm_number}{year}_fcst_latest.zip',
                    f'al{storm_number}{year}_best_track.zip',
                    f'gtwo_shapefiles.zip',
                    f'wsp_120hrhalfDeg_latest.zip',
                    f'wsp_120hr5km_latest.zip'
                   ]
    urls         = [
                    f'https://www.nhc.noaa.gov/gis/forecast/archive',
                    f'https://www.nhc.noaa.gov/gis/forecast/archive',
                    f'https://www.nhc.noaa.gov/gis/best_track',
                    f'https://www.nhc.noaa.gov/xgtwo',
                    f'https://www.nhc.noaa.gov/gis/forecast/archive',
                    f'https://www.nhc.noaa.gov/gis/forecast/archive'
                   ]
    gdf_names   =  {
                    file_names[0] : ['track_line_gdf', 'cone_gdf', 'points_gdf'],
                    file_names[1] : ['init_radii_gdf', 'fcst_radii_gdf'],
                    file_names[2] : ['best_track_points_gdf', 'best_track_line_gdf', 'best_track_radii_gdf', 'best_track_swath_gdf'],
                    file_names[3] : ['gtwo_areas_gdf', 'gtwo_lines_gdf', 'gtwo_points_gdf'],
                    file_names[4] : ['wsp_34_gdf_points', 'wsp_50_gdf_points', 'wsp_64_gdf_points'],
                    file_names[5] : ['wsp_34_gdf_polygons', 'wsp_50_gdf_polygons', 'wsp_64_gdf_polygons']
                   }
    return file_names, urls, gdf_names
nhc_fnames()

(['al122021_5day_latest.zip',
  'al122021_fcst_latest.zip',
  'al122021_best_track.zip',
  'gtwo_shapefiles.zip',
  'wsp_120hrhalfDeg_latest.zip',
  'wsp_120hr5km_latest.zip'],
 ['https://www.nhc.noaa.gov/gis/forecast/archive',
  'https://www.nhc.noaa.gov/gis/forecast/archive',
  'https://www.nhc.noaa.gov/gis/best_track',
  'https://www.nhc.noaa.gov/xgtwo',
  'https://www.nhc.noaa.gov/gis/forecast/archive',
  'https://www.nhc.noaa.gov/gis/forecast/archive'],
 {'al122021_5day_latest.zip': ['track_line_gdf', 'cone_gdf', 'points_gdf'],
  'al122021_fcst_latest.zip': ['init_radii_gdf', 'fcst_radii_gdf'],
  'al122021_best_track.zip': ['best_track_points_gdf',
   'best_track_line_gdf',
   'best_track_radii_gdf',
   'best_track_swath_gdf'],
  'gtwo_shapefiles.zip': ['gtwo_areas_gdf',
   'gtwo_lines_gdf',
   'gtwo_points_gdf'],
  'wsp_120hrhalfDeg_latest.zip': ['wsp_34_gdf_points',
   'wsp_50_gdf_points',
   'wsp_64_gdf_points'],
  'wsp_120hr5km_latest.zip': ['wsp_34_gdf_polygons',
   'wsp_50_g

In [20]:
def nhc_gis_downloader(file_names, urls, gdf_names):

    for idx, file_name in enumerate(file_names):
        url = f'{urls[idx]}/{file_name}'
        r = requests.get(url)
        with open(f'nhc_latest/{file_name}', 'wb') as code:
            code.write(r.content) 
    _  = [0, 3, 4, 5]  
    __ = [2, 7, 12]  
    for idx, file_name in enumerate([file_names[0],  file_names[3], 
                                    file_names[4],  file_names[5]]): 
        with ZipFile(f'nhc_latest/{file_name}', 'r') as zip_file_name:
            for idx2 in range(len(gdf_names[file_names[_[idx]]])):
                gdf_names[file_names[_[idx]]][idx2]  = geopandas.read_file(
                f'zip://./nhc_latest/{file_name}!{zip_file_name.namelist()[__[idx2]]}')
    _  = [0,5]                                 
    with ZipFile(f'nhc_latest/{file_names[1]}', 'r') as zip_file_name:
        for idx in range(2):
            gdf_names[file_names[1]][idx]  = geopandas.read_file(
            f'zip://./nhc_latest/{file_names[1]}!{zip_file_name.namelist()[_[idx]]}')
    _  = [2, 7, 13, 16]                                 
    with ZipFile(f'nhc_latest/{file_names[2]}', 'r') as zip_file_name:
        for idx in range(4):
            gdf_names[file_names[2]][idx]  = geopandas.read_file(
            f'zip://./nhc_latest/{file_names[2]}!{zip_file_name.namelist()[_[idx]]}')
    return gdf_names

file_names = nhc_fnames()[0]
urls       = nhc_fnames()[1]
gdf_names  = nhc_fnames()[2]

nhc_gis_downloader(file_names, urls, gdf_names)[nhc_fnames()[0][1]][0]



,RADII,STORMID,BASIN,STORMNUM,ADVNUM,VALIDTIME,SYNOPTIME,TIMEZONE,TAU,NE,SE,SW,NW,geometry
0,34.0,al122021,al,12.0,25,2021090621,2021090618,UTC,0.0,160.0,150.0,100.0,140.0,"POLYGON ((-53.87435 25.16180, -53.82305 25.160..."
1,50.0,al122021,al,12.0,25,2021090621,2021090618,UTC,0.0,90.0,80.0,70.0,80.0,"POLYGON ((-53.88570 23.99726, -53.85711 23.996..."
2,64.0,al122021,al,12.0,25,2021090621,2021090618,UTC,0.0,60.0,50.0,40.0,50.0,"POLYGON ((-53.89051 23.49818, -53.87151 23.497..."
3,34.0,al122021,al,12.0,25,2021090706,2021090618,UTC,12.0,160.0,150.0,100.0,140.0,"POLYGON ((-54.97416 26.06180, -54.92246 26.060..."
4,50.0,al122021,al,12.0,25,2021090706,2021090618,UTC,12.0,90.0,80.0,70.0,80.0,"POLYGON ((-54.98560 24.89726, -54.95680 24.896..."
5,64.0,al122021,al,12.0,25,2021090706,2021090618,UTC,12.0,60.0,50.0,40.0,50.0,"POLYGON ((-54.99044 24.39818, -54.97131 24.397..."
6,34.0,al122021,al,12.0,25,2021090718,2021090618,UTC,24.0,160.0,150.0,110.0,140.0,"POLYGON ((-56.27381 27.56180, -56.22143 27.560..."
7,50.0,al122021,al,12.0,25,2021090718,2021090618,UTC,24.0,90.0,90.0,70.0,80.0,"POLYGON ((-56.28542 26.39726, -56.25625 26.396..."
8,64.0,al122021,al,12.0,25,2021090718,2021090618,UTC,24.0,60.0,50.0,40.0,50.0,"POLYGON ((-56.29032 25.89818, -56.27096 25.897..."
9,34.0,al122021,al,12.0,25,2021090806,2021090618,UTC,36.0,160.0,160.0,120.0,140.0,"POLYGON ((-57.57341 29.16180, -57.52024 29.160..."


In [43]:
class NhcDownloaderBot:
    def __init__(self, storm_number, year):
        self.file_names   = file_names = [
                    f'al{storm_number}{year}_5day_latest.zip',
                    f'al{storm_number}{year}_fcst_latest.zip',
                    f'al{storm_number}{year}_best_track.zip',
                    f'gtwo_shapefiles.zip',
                    f'wsp_120hrhalfDeg_latest.zip',
                    f'wsp_120hr5km_latest.zip'
                   ]
        self.urls         = urls = [
                    f'https://www.nhc.noaa.gov/gis/forecast/archive',
                    f'https://www.nhc.noaa.gov/gis/forecast/archive',
                    f'https://www.nhc.noaa.gov/gis/best_track',
                    f'https://www.nhc.noaa.gov/xgtwo',
                    f'https://www.nhc.noaa.gov/gis/forecast/archive',
                    f'https://www.nhc.noaa.gov/gis/forecast/archive'
                   ]
        self.gdf_names   =  gdf_names = {
                    file_names[0] : ['track_line_gdf', 'cone_gdf', 'points_gdf'],
                    file_names[1] : ['init_radii_gdf', 'fcst_radii_gdf'],
                    file_names[2] : ['best_track_points_gdf', 'best_track_line_gdf', 'best_track_radii_gdf', 'best_track_swath_gdf'],
                    file_names[3] : ['gtwo_areas_gdf', 'gtwo_lines_gdf', 'gtwo_points_gdf'],
                    file_names[4] : ['wsp_34_gdf_points', 'wsp_50_gdf_points', 'wsp_64_gdf_points'],
                    file_names[5] : ['wsp_34_gdf_polygons', 'wsp_50_gdf_polygons', 'wsp_64_gdf_polygons']
                   }
    def nhc_gis_downloader(self, file_names, urls, gdf_names):

        for idx, file_name in enumerate(file_names):
            url = f'{urls[idx]}/{file_name}'
            r = requests.get(url)
            with open(f'nhc_latest/{file_name}', 'wb') as code:
                code.write(r.content) 
        _  = [0, 3, 4, 5]  
        __ = [2, 7, 12]  
        for idx, file_name in enumerate([file_names[0],  file_names[3], 
                                        file_names[4],  file_names[5]]): 
            with ZipFile(f'nhc_latest/{file_name}', 'r') as zip_file_name:
                for idx2 in range(len(gdf_names[file_names[_[idx]]])):
                    gdf_names[file_names[_[idx]]][idx2]  = geopandas.read_file(
                    f'zip://./nhc_latest/{file_name}!{zip_file_name.namelist()[__[idx2]]}')
        _  = [0,5]                                 
        with ZipFile(f'nhc_latest/{file_names[1]}', 'r') as zip_file_name:
                for idx in range(2):
                    gdf_names[file_names[1]][idx]  = geopandas.read_file(
                    f'zip://./nhc_latest/{file_names[1]}!{zip_file_name.namelist()[_[idx]]}')
        _  = [2, 7, 13, 16]                                 
        with ZipFile(f'nhc_latest/{file_names[2]}', 'r') as zip_file_name:
            for idx in range(4):
                gdf_names[file_names[2]][idx]  = geopandas.read_file(
                f'zip://./nhc_latest/{file_names[2]}!{zip_file_name.namelist()[_[idx]]}')
        return gdf_names
    

# file_names = nhc_fnames()[0]
# urls       = nhc_fnames()[1]
# gdf_names  = nhc_fnames()[2]

# nhc_gis_downloader(file_names, urls, gdf_names)[nhc_fnames()[0][1]][0]
        

In [36]:
_ = NhcDownloaderBot(12, 2021)

_.nhc_gis_downloader(_.file_names, _.urls, _.gdf_names)[nhc_fnames()[0][0]][0]

,STORMNAME,STORMTYPE,ADVDATE,ADVISNUM,STORMNUM,FCSTPRD,BASIN,geometry
0,Larry,HU,500 PM AST Mon Sep 06 2021,25,12.0,120.0,AL,"LINESTRING (-53.90000 22.50000, -55.00000 23.4..."


In [45]:
def gdf_file(): 
    _ = NhcDownloaderBot(12, 2021)
    return _.nhc_gis_downloader(_.file_names, _.urls, _.gdf_names)[nhc_fnames()[0][0]][0]